# 2.4 신경망의 엔진: 그래디언트 기반 최적화
- `output = relu(dot(W, input) + b)`, `W`, `b`를 **가중치** 또는 **훈련되는 파라미터**(trainable parameter)라고 부름
- 초기에는 가중치 행렬이 작은 난수로 채워져 있음 (**무작위 난수화**(random initialization) 단계)
- 피드백 신호에 기초하여 가중치가 점진적으로 조정되는데 이 과정(**훈련**)이 머신 러닝 학습의 핵심
- **훈련 반복 루프**(training loop)
    1. 훈련 샘플 `x`와 이에 상응하는 타깃 `y`의 배치를 추출
    2. `x`를 사용하여 네트워크를 실행하고(정방향 패스(forward pass) 단계), 예측 `y_pred`를 구함
    3. `y_pred`와 `y`의 차이를 측정하여 이 배치에 대한 네트워크의 손실 계산
    4. 배치에 대한 손실이 조금 감소되도록 네트워크의 모든 가중치를 업데이트
- 어려운 것은 가중치를 업데이트하는 4단계
- 관심 있는 하나의 가중치를 제외하고 모두 고정시킨 후 그 가중치를 바꿔가며 손실이 줄어드는 것을 확인
    - 이 방식은 모든 가중치 행렬의 원소마다 두 번의 비용이 큰 정방향 패스를 계산해야 하므로 엄청나게 비효율적
- 신경망에 사용된 모든 연산이 **미분 가능**(differentiable)하다는 장점을 사용하여 네트워크 가중치에 대한 손실의 **그래디언트**(gradient)를 계산하는 것이 훨씬 더 좋은 방법

## 2.4.1 변화율이란?
- y=f(x)의 한 점 x에서 아주 작은 값 epsilon_x만큼 움직였을 때 y의 변화량을 a\*epsilon_x로 볼 수 있으며, 이 때의 기울기를 그 점에서 f의 **변화율**(derivative)라고 함

## 2.4.2 텐서 연산의 변화율: 그래디언트
- **그래디언트** : 텐서 연산의 변화율
- `y_pred = dot(W, x)`, `loss_value = loss(y_pred, y)`일 때, 입력 데이터 `x, y`가 고정되어 있다면 `loss_value = f(W)`로 볼 수 있음
- `W`의 현재값 `W0`에서 `f`의 변화율은 `gradient(f)(W0)`
    - 이 텐서의 각 원소 `gradient(f)(W0)[i, j]`는 `W0[i, j]`를 변경했을 때 `loss_value`가 바뀌는 방향과 크기를 나타냄

## 2.4.3 확률적 경사 하강법
- `gradient(f)(W) = 0`을 풀면 해결되지만 파라미터의 개수가 매우 많기 때문에 해석적으로 해결하는 것이 어려움
- **미니 배치 확률적 경사 하강법**(mini-batch stochastic gradient descent)
    1. 훈련 샘플 배치 `x`와 이에 상응하는 타깃 `y`를 추출
    2. `x`로 네트워크를 실행하고 예측 `y_pred`를 구함
    3. 이 배치에서 `y_pred`와 `y` 사이의 오차를 측정하여 네트워크의 손실을 계산
    4. 네트워크의 파라미터에 대한 손실 함수의 그래디언트를 계산(**역방향 패스**(backward pass))
    5. 그래디언트 반대 방향으로 파라미터를 조금 이동
        - `W -= step * gradient`
- `step`이 너무 작으면 너무 많은 반복이 필요하고 지역 최솟값(local minimum)에 갇힐 수 있음
- `step`이 너무 크면 완전히 임의의 위치로 이동시킬 수 있음
- 미니 배치 SGD 알고리즘 중 하나는 반복마다 하나의 샘플과 하나의 타깃을 뽑는 것
- 가용한 모든 데이터를 사용하여 반복하는 것을 **배치 SGD**(batch SGD)라고 하며 더 정확하게 업데이트되지만 더 많은 비용이 듬
- 1D, 2D 손실 함수 표면을 따라 진행하는 경사 하강법을 시각화해볼 수 있으나 그 이상은 불가능
    - 그렇기 때문에 저차원 표현으로 얻은 직관이 실전과 항상 맞지는 않는다는 것을 유념
    - 대표적으로 신경망 알고리즘이 지역 최솟값에 쉽게 갇힐 것으로 생각했으나 고차원 공간에서는 대부분 안장점(saddle point)으로 나타나고 지역 최솟값은 매우 드뭄
- 업데이트할 다음 가중치를 계산할 때 현재 그래디언트 값만 보지 않고 이전에 업데이트된 가중치를 여러 가지 다른 방식으로 고려하는 SGD 변종이 있음
- 이런 변종들을 **최적화 방법**(optimization method) 또는 **옵티마이저**라고 부름
    - 모멘텀을 사용한 SGD, Adagrad, RMSProp 등
- 단순한 구현 예 (모멘텀을 두 번 반복하는 알고리즘인 네스테로프 모멘텀(Nesterov Momentum)을 구현한 것으로 일반적으로 `momentum` 값은 0.9 정도를 많이 사용)

```python
past_velocity = 0
momentum = 0.1    # 모멘텀 상수
while loss > 0.01:    # 최적화 반복 루프
    w, loss, gradient = get_current_parameters()
    velocity = momentum * past_velocity - learning_rate * gradient
    w = w + momentum * velocity - learning_rate * gradient
    past_velocity = velocity
    update_parameter(w)
```

## 2.4.4 변화율 연결: 역전파 알고리즘
- 신경망은 많은 텐서 연산으로 구성되어 있고 이 연산들의 변화율은 간단하며 이미 잘 알려져 있음
- 예시 : 3개의 텐서 연산 `a, b, c`와 가중치 행렬 `W1, W2, W3`로 구성된 네트워크 `f`
    - `f(W1, W2, W3) = a(W1, b(W2, c(W3)))`
    - **연쇄 법칙**(chain rule)을 사용, 신경망의 그래디언트를 계산하는 **역전파**(Backpropagation) 알고리즘(**후진 모드 자동 미분**(reversed-mode automatic differentiation)이라고도 부름) 탄생
    - 역전파는 최종 손실 값에서부터 시작, 손실 값에 각 파라미터가 기여한 정도를 계산하기 위해 연쇄 법칙을 적용하여 최상위 층에서 하위 층까지 거꾸로 진행
- 요즘 또는 향후 몇 년 간 텐서플로처럼 **기호 미분**(symbolic differentiation)이 가능한 최신 프레임워크를 사용하여 신경망을 구현할 것
    - 변화율이 알려진 연산들로 연결되어 있으면 연쇄 법칙을 적용하여 네트워크 파라미터와 그래디언트 값을 매핑하는 그래디언트 함수를 계산할 수 있다는 의미